In [19]:
import numpy as np
import matplotlib.pyplot as plt
import re

# Convert the list of instructions from its input form 'U123' to a list
# of points in the cartesian plane visited by the wire
def to_cartesian(wirepath):
    
    # Declare cartesian list to store positions in cartesian coordinates
    # starting from the origin (the central port)
    cartesian = [[0, 0]]
    
    # Length of the instruction set
    length = len(wirepath)
    
    for i in range(1, length+1):
        
        # Split the instruction, e.g.: 'U123' -> ['U', '1', '2', '3']
        a = list(wirepath[i-1])
        
        # First element is the direction to move in
        direction = a[0]
        
        # Removing the character, the remaining information is an int that
        # instructs on how much to move in the said direction
        value = int(wirepath[i-1].replace(direction, ''))
        
        # Append to 'cartesian' the new cartesian coordinates resulting from moving
        # from the last position following the instruction
        if direction == 'U':
            cartesian.append([cartesian[i-1][0], cartesian[i-1][1] + value])
        elif direction == 'D':
            cartesian.append([cartesian[i-1][0], cartesian[i-1][1] - value])
        elif direction == 'R':
            cartesian.append([cartesian[i-1][0] + value, cartesian[i-1][1]])
        elif direction == 'L':
            cartesian.append([cartesian[i-1][0] - value, cartesian[i-1][1]])
      
    # Return the new list created in this way
    return cartesian
     
    
# This function calculates the Manhattan distance of a point
# from the origin of the coordinate grid
def Manhattan(point):
    
    return abs(point[0]) + abs(point[1])


# Length of the wire. The 'stop' parameter specify a point at which
# the calculation must end. By default it is [0.5, 0.5], so it returns the length
# of the whole wire (since the path has only integer points).
def wire_length(wirepath, stop=[0.5, 0.5]):
    
    prev = np.array([0, 0])
    distance = 0
    
    for point in wirepath:
        diff = point - prev
        distance += abs(diff[0]) + abs(diff[1])
        prev = point
        if np.all(point == stop):
            break
        
    return distance


# Function that fills instruction lists into its atomic instructions
def fill_instruction_list(instr_list):
    
    filled = []
    
    for instruction in instr_list:
        direction = re.findall('[A-Z]', instruction)[0]
        value = int(re.findall('[0-9]+', instruction)[0])
    
        for i in range(value):
            # Move 1 step in the specified direction
            filled.append(direction+'1')
    
    return filled


# Function that creates a unique hash string from a pair of cartesia points as:
# (12, 34) --> x12y34. Modified from Part1: this function does not act on lists but on single values
# either 
def cartesian_hash(expr, option = 0):
    
    if option == 'r':
        
        elem = list(expr.replace('x', '').split('y'))
        hashed = [int(x) for x in elem]
        
    else:
        
        hashed = 'x'+str(expr[0])+'y'+str(expr[1])
    
    return hashed
    

In [20]:
# Load instruction from file
with open("input.txt", 'r') as infile:
    
    wire_1 = infile.readline().replace('\n', '').split(',')
    wire_2 = infile.readline().replace('\n', '').split(',')
    
# Extend the instruction sets and store cartesian form           
wire_1_ext = np.array(to_cartesian(fill_instruction_list(wire_1)))
wire_2_ext = np.array(to_cartesian(fill_instruction_list(wire_2)))

print(wire_length(wire_1_ext), wire_length(wire_2_ext))

147638 150424


In [15]:
# Initialize the two sets
wire_1_set = set([cartesian_hash(x) for x in wire_1_ext])
wire_2_set = set([cartesian_hash(x) for x in wire_2_ext])

# Find the intersections
set_crosses = []

for point in wire_1_set:
    if point in wire_2_set:
        set_crosses.append(point)
    
# We must also convert back to the cartesian form from the hashed form
crosses = np.array([cartesian_hash(x, 'r') for x in set_crosses])

In [28]:
# Find the intersection points which is closest to the origin following the wires' paths.

# Set a starting value (bigger than all the others)
shortest = wire_length(wire_1_ext)

for cross in crosses:
    # The origin doesn't count
    if np.all(cross == [0, 0]):
        continue
        
    # Distance along the wires to the intersection point
    wire_1_steps = wire_length(wire_1_ext, cross)
    wire_2_steps = wire_length(wire_2_ext, cross)
    to_get_there = wire_1_steps + wire_2_steps
    if to_get_there <= shortest:
        shortest = to_get_there
                
print(shortest)

107036
